In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import numpy as np
import pandas as pd

tweets = []
with open('trainingdata-all-annotations.txt','r', encoding = 'iso-8859-1') as f:
    while True:
        line = f.readline()
        if not line:
            break
        tweets.append(line.split('\t'))
        
mat = np.array(tweets)
mat = np.delete(mat, 0, axis=0)

df_training = pd.DataFrame(mat, columns = ['ID','Target','Tweet','Stance','Opinion towards', 'Sentiment'])
df_training = df_training.drop(['ID','Opinion towards', 'Sentiment'], axis=1)

print(df_training)

                        Target  \
0                      Atheism   
1                      Atheism   
2                      Atheism   
3                      Atheism   
4                      Atheism   
...                        ...   
2809  Legalization of Abortion   
2810  Legalization of Abortion   
2811  Legalization of Abortion   
2812  Legalization of Abortion   
2813  Legalization of Abortion   

                                                  Tweet   Stance  
0     dear lord thank u for all of ur blessings forg...  AGAINST  
1     Blessed are the peacemakers, for they shall be...  AGAINST  
2     I am not conformed to this world. I am transfo...  AGAINST  
3     Salah should be prayed with #focus and #unders...  AGAINST  
4     And stay in your houses and do not display you...  AGAINST  
...                                                 ...      ...  
2809  There's a law protecting unborn eagles, but no...  AGAINST  
2810  I am 1 in 3... I have had an abortion #Abortio...

In [3]:
tweets = []
with open('testdata-taskA-all-annotations.txt','r', encoding = 'iso-8859-1') as f:
    while True:
        line = f.readline()
        if not line:
            break
        tweets.append(line.split('\t'))
        
mat = np.array(tweets)
mat = np.delete(mat, 0, axis=0)

df_testing = pd.DataFrame(mat, columns = ['ID','Target','Tweet','Stance','Opinion towards', 'Sentiment'])
df_testing = df_testing.drop(['ID','Opinion towards', 'Sentiment'], axis=1)

print(df_testing)

                        Target  \
0                      Atheism   
1                      Atheism   
2                      Atheism   
3                      Atheism   
4                      Atheism   
...                        ...   
1244  Legalization of Abortion   
1245  Legalization of Abortion   
1246  Legalization of Abortion   
1247  Legalization of Abortion   
1248  Legalization of Abortion   

                                                  Tweet   Stance  
0     He who exalts himself shall      be humbled; a...  AGAINST  
1     RT @prayerbullets: I remove Nehushtan -previou...  AGAINST  
2     @Brainman365 @heidtjj @BenjaminLives I have so...  AGAINST  
3     #God is utterly powerless without Human interv...  AGAINST  
4     @David_Cameron   Miracles of #Multiculturalism...  AGAINST  
...                                                 ...      ...  
1244  @MetalheadMonty @tom_six I followed him before...     NONE  
1245  For he who avenges blood remembers, he does no...

In [4]:
print(df_testing['Stance'])

0       AGAINST
1       AGAINST
2       AGAINST
3       AGAINST
4       AGAINST
         ...   
1244       NONE
1245    AGAINST
1246    AGAINST
1247    AGAINST
1248    AGAINST
Name: Stance, Length: 1249, dtype: object


In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()

le1.fit(df_training['Stance'])
df_training['Stance'] = le1.transform(df_training['Stance'])
df_testing['Stance'] = le1.transform(df_testing['Stance'])

le2.fit(df_training['Target'])
df_training['Target'] = le2.transform(df_training['Target'])
df_testing['Target'] = le2.transform(df_testing['Target'])
print(len(df_training['Target']))

df_training.head()

2814


,Target,Tweet,Stance
0,0,dear lord thank u for all of ur blessings forg...,0
1,0,"Blessed are the peacemakers, for they shall be...",0
2,0,I am not conformed to this world. I am transfo...,0
3,0,Salah should be prayed with #focus and #unders...,0
4,0,And stay in your houses and do not display you...,0


In [6]:
from sklearn.preprocessing import MinMaxScaler

cols_to_norm = ['Target']
df_training[cols_to_norm] = MinMaxScaler().fit_transform(df_training[cols_to_norm])
df_testing[cols_to_norm] = MinMaxScaler().fit_transform(df_testing[cols_to_norm])

In [7]:
from nltk.stem.porter import PorterStemmer
import re

porter_stemmer = PorterStemmer()

def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [8]:
unique_words = set();
for string in df_training['Tweet']:
    words = stemming_tokenizer(string)
    unique_words.update(words)
for string in df_testing['Tweet']:
    words = stemming_tokenizer(string)
    unique_words.update(words)
print(len(unique_words))

9793


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer( stop_words = 'english', tokenizer=stemming_tokenizer, vocabulary=unique_words)

# Compute the TF-IDF matrix
training_tfidf = vectorizer.fit_transform(df_training['Tweet'])
testing_tfidf = vectorizer.fit_transform(df_testing['Tweet'])

X_train = pd.DataFrame(training_tfidf.toarray(), columns=vectorizer.get_feature_names())
X_train['Target'] = df_training['Target']
X_train.head()

/Users/caryslekander/opt/anaconda3/envs/python38env/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:382: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', '

,-,--,-------,--isaiah,--ronald,-2,-act,-babi,-blais,-but,...,zerofootprint,zim,zinta,zip,zmanoj,zoe,zombieneith,zommer,zubair,Target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_test = pd.DataFrame(testing_tfidf.toarray(), columns=vectorizer.get_feature_names())
X_test['Target'] = df_testing['Target']
X_test.head()

,-,--,-------,--isaiah,--ronald,-2,-act,-babi,-blais,-but,...,zerofootprint,zim,zinta,zip,zmanoj,zoe,zombieneith,zommer,zubair,Target
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.297131,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y_train = df_training.Stance.astype(np.int32)
y_test = df_testing.Stance.astype(np.int32)

In [12]:
model = LogisticRegression(solver='liblinear', random_state=0, max_iter = 500)

In [13]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
model.predict_proba(X_test)

array([[0.52861993, 0.22510041, 0.24627966],
       [0.66088052, 0.19601606, 0.14310342],
       [0.42304308, 0.27075295, 0.30620397],
       ...,
       [0.6503215 , 0.16831224, 0.18136626],
       [0.49507971, 0.16034102, 0.34457928],
       [0.56499471, 0.13069243, 0.30431286]])

In [15]:
y_hat = model.predict(X_test)

In [16]:
model.score(X_test, y_test)

0.6325060048038431

In [28]:
results = pd.DataFrame()
results['Target'] = X_test['Target']
results['Actual Stance'] = le1.inverse_transform(y_test)
results['Predicted Stance'] = le1.inverse_transform(y_hat)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(results)

      Target Actual Stance Predicted Stance
0       0.00       AGAINST          AGAINST
1       0.00       AGAINST          AGAINST
2       0.00       AGAINST          AGAINST
3       0.00       AGAINST          AGAINST
4       0.00       AGAINST          AGAINST
5       0.00       AGAINST             NONE
6       0.00       AGAINST            FAVOR
7       0.00       AGAINST          AGAINST
8       0.00       AGAINST             NONE
9       0.00       AGAINST          AGAINST
10      0.00       AGAINST             NONE
11      0.00       AGAINST            FAVOR
12      0.00       AGAINST          AGAINST
13      0.00       AGAINST            FAVOR
14      0.00       AGAINST          AGAINST
15      0.00       AGAINST          AGAINST
16      0.00       AGAINST          AGAINST
17      0.00       AGAINST          AGAINST
18      0.00       AGAINST          AGAINST
19      0.00       AGAINST          AGAINST
20      0.00       AGAINST          AGAINST
21      0.00       AGAINST      

In [30]:
correct_favor = len(results[(results['Actual Stance']=='FAVOR') & (results['Predicted Stance']=='FAVOR')])
favor_perct = correct_favor / len(results[(results['Actual Stance']=='FAVOR')])
correct_against = len(results[(results['Actual Stance']=='AGAINST') & (results['Predicted Stance']=='AGAINST')])
against_perct = correct_against / len(results[(results['Actual Stance']=='AGAINST')])
correct_none = len(results[(results['Actual Stance']=='NONE') & (results['Predicted Stance']=='NONE')])
none_perct = correct_none / len(results[(results['Actual Stance']=='NONE')])

print(f"F: {correct_favor},  {favor_perct*100}%")
print(f"A: {correct_against},  {against_perct*100}%")
print(f"N: {correct_none},  {none_perct*100}%")


F: 113,  37.17105263157895%
A: 623,  87.13286713286713%
N: 54,  23.47826086956522%


In [34]:
correct_favor_1 = len(results[(results['Actual Stance']=='FAVOR') & (results['Predicted Stance']=='FAVOR') & (results['Target']=='0.0')])
favor_perct_1 = correct_favor / len(results[(results['Actual Stance']=='FAVOR') & (results['Target']=='0.00')])
correct_against_1 = len(results[(results['Actual Stance']=='AGAINST') & (results['Predicted Stance']=='AGAINST') & (results['Target']=='0.0')])
against_perct_1 = correct_against / len(results[(results['Actual Stance']=='AGAINST') & (results['Target']=='0.0')])
correct_none_1 = len(results[(results['Actual Stance']=='NONE') & (results['Predicted Stance']=='NONE') & (results['Target']=='0.0')])
none_perct_1 = correct_none / len(results[(results['Actual Stance']=='NONE') & (results['Target']=='0.0')])

correct_favor_2 = len(results[(results['Actual Stance']=='FAVOR') & (results['Predicted Stance']=='FAVOR') & (results['Target']=='0.25')])
favor_perct_2 = correct_favor / len(results[(results['Actual Stance']=='FAVOR') & (results['Target']=='0.25')])
correct_against_2 = len(results[(results['Actual Stance']=='AGAINST') & (results['Predicted Stance']=='AGAINST') & (results['Target']=='0.25')])
against_perct_2 = correct_against / len(results[(results['Actual Stance']=='AGAINST') & (results['Target']=='0.25')])
correct_none_2 = len(results[(results['Actual Stance']=='NONE') & (results['Predicted Stance']=='NONE') & (results['Target']=='0.25')])
none_perct_2 = correct_none / len(results[(results['Actual Stance']=='NONE') & (results['Target']=='0.25')])

correct_favor_3 = len(results[(results['Actual Stance']=='FAVOR') & (results['Predicted Stance']=='FAVOR') & (results['Target']=='0.5')])
favor_perct_3 = correct_favor / len(results[(results['Actual Stance']=='FAVOR') & (results['Target']=='0.5')])
correct_against_3 = len(results[(results['Actual Stance']=='AGAINST') & (results['Predicted Stance']=='AGAINST') & (results['Target']=='0.5')])
against_perct_3 = correct_against / len(results[(results['Actual Stance']=='AGAINST') & (results['Target']=='0.5')])
correct_none_3 = len(results[(results['Actual Stance']=='NONE') & (results['Predicted Stance']=='NONE') & (results['Target']=='0.5')])
none_perct_3 = correct_none / len(results[(results['Actual Stance']=='NONE') & (results['Target']=='0.5')])

correct_favor_4 = len(results[(results['Actual Stance']=='FAVOR') & (results['Predicted Stance']=='FAVOR') & (results['Target']=='0.75')])
favor_perct_4 = correct_favor / len(results[(results['Actual Stance']=='FAVOR') & (results['Target']=='0.75')])
correct_against_4 = len(results[(results['Actual Stance']=='AGAINST') & (results['Predicted Stance']=='AGAINST') & (results['Target']=='0.75')])
against_perct_4 = correct_against / len(results[(results['Actual Stance']=='AGAINST') & (results['Target']=='0.75')])
correct_none_4 = len(results[(results['Actual Stance']=='NONE') & (results['Predicted Stance']=='NONE') & (results['Target']=='0.75')])
none_perct_4 = correct_none / len(results[(results['Actual Stance']=='NONE') & (results['Target']=='0.75')])

correct_favor_5 = len(results[(results['Actual Stance']=='FAVOR') & (results['Predicted Stance']=='FAVOR') & (results['Target']=='1.0')])
favor_perct_5 = correct_favor / len(results[(results['Actual Stance']=='FAVOR') & (results['Target']=='1.0')])
correct_against_5 = len(results[(results['Actual Stance']=='AGAINST') & (results['Predicted Stance']=='AGAINST') & (results['Target']=='1.0')])
against_perct_5 = correct_against / len(results[(results['Actual Stance']=='AGAINST') & (results['Target']=='1.0')])
correct_none_5 = len(results[(results['Actual Stance']=='NONE') & (results['Predicted Stance']=='NONE') & (results['Target']=='1.0')])
none_perct_5 = correct_none / len(results[(results['Actual Stance']=='NONE') & (results['Target']=='1.0')])

print("Topic 1")
print(f"F: {correct_favor_1},  {favor_perct_1*100}%")
print(f"A: {correct_against_1},  {against_perct_1*100}%")
print(f"N: {correct_none_1},  {none_perct_1*100}%")

print("Topic 2")
print(f"F: {correct_favor_2},  {favor_perct_2*100}%")
print(f"A: {correct_against_2},  {against_perct_2*100}%")
print(f"N: {correct_none_2},  {none_perct_2*100}%")

print("Topic 3")
print(f"F: {correct_favor_3},  {favor_perct_3*100}%")
print(f"A: {correct_against_3},  {against_perct_3*100}%")
print(f"N: {correct_none_3},  {none_perct_3*100}%")

print("Topic 4")
print(f"F: {correct_favor_4},  {favor_perct_4*100}%")
print(f"A: {correct_against_4},  {against_perct_4*100}%")
print(f"N: {correct_none_4},  {none_perct_4*100}%")

print("Topic 5")
print(f"F: {correct_favor_5},  {favor_perct_5*100}%")
print(f"A: {correct_against_5},  {against_perct_5*100}%")
print(f"N: {correct_none_5},  {none_perct_5*100}%")


ZeroDivisionError: division by zero

In [18]:
from sklearn import metrics as mt

print(mt.confusion_matrix(y_test,y_hat))
print(mt.classification_report(y_test,y_hat))

[[623  60  32]
 [173 113  18]
 [160  16  54]]
              precision    recall  f1-score   support

           0       0.65      0.87      0.75       715
           1       0.60      0.37      0.46       304
           2       0.52      0.23      0.32       230

    accuracy                           0.63      1249
   macro avg       0.59      0.49      0.51      1249
weighted avg       0.61      0.63      0.60      1249



In [19]:
print(le1.inverse_transform(y_test))

['AGAINST' 'AGAINST' 'AGAINST' ... 'AGAINST' 'AGAINST' 'AGAINST']


{}